In [ ]:
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.layers.experimental import preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time

In [ ]:
# Load Dataset and split into partitions
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Create a validation set from training images
validation_images = train_images[:5000]
validation_labels = train_labels[:5000]

In [ ]:
# Analyze the stuctural properties of the training dataset
print("Shape of training images", train_images.shape)
print("Shape of training labels", train_labels.shape)
print("Number of training labels",  train_labels[0])
print("\n")

# Alternatively you can use the Python len() method
print("Number of training images", len(train_images))
print("Number of training labels", len(train_labels))
print("\n")

# Analyze the stuctural properties of the test dataset
print("Shape of images", test_images.shape)
print("Shape of labels", test_labels.shape)
print("Number of labels",  test_labels[0])
print("\n")

# Alternatively you can use the Python len() method
print("Number of test images", len(test_images))
print("Number of test labels", len(test_labels))

In [ ]:
# Example of training label content
print(train_labels[0])
print("Minimum value of training labels", train_labels.min())
print("Maximum value of training labels", train_labels.max())

In [ ]:
# Content of training images before normalisation
print(train_images[0])
print("Minimum value of training images", train_images.min())
print("Maximum value of training images", train_images.max())

In [ ]:
# TODO: Test that this works on TensorFlow version 2.1+
# Normalizing trianing images through keras preprocessing layers
normalization_layer = preprocessing.Normalization()
normalization_layer.adapt(train_images)
train_images = normalization_layer(train_images)

# Normalize test images
normalization_layer.adapt(test_images)
test_images = normalization_layer(test_images)

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
# Normalise training and test images
train_images = train_images / 255.0
test_images = test_images / 255.0
validation_images = validation_images / 255.0


In [ ]:
# Content of training images after normalisation
print("Minimum value of training images", train_images.min())
print("Maximum value of training images", train_images.max())

In [ ]:
# Label	Description
# 0	T-shirt/top
# 1	Trouser
# 2	Pullover
# 3	Dress
# 4	Coat
# 5	Sandal
# 6	Shirt
# 7	Sneaker
# 8	Bag
# 9	Ankle boot
fashion_mnist_class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
fashion_mnist_class_names[train_labels[2]]

In [ ]:
# Visualising data
plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(5, 4, i+1)
    plt.axis("off")
    plt.title(fashion_mnist_class_names[train_labels[i]])
    plt.imshow(train_images[i])
plt.show()

In [ ]:
# Classification MLP(Multilayer perceptron) with three hidden layers
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(500, activation=keras.activations.relu),
    keras.layers.Dense(250, activation=keras.activations.relu),
    keras.layers.Dense(100, activation=keras.activations.relu),
    keras.layers.Dense(10, activation=keras.activations.softmax)
])

In [ ]:
model.summary()

In [ ]:
# Model compilation and initialisation of optimizer
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [ ]:
# Configure TensorBoard storage location
root_logdir = os.path.join(os.curdir, "runs")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
# Train model
model.fit(train_images, train_labels, epochs=60, validation_data=(validation_images, validation_labels), callbacks=[tensorboard_cb, early_stopping_cb])

In [ ]:
model_evaluation_results = model.evaluate(test_images, test_labels, verbose=2)
print("The test loss is", model_evaluation_results[0])
print("The test accuracy is", model_evaluation_results[1])

In [ ]:
# Prediction on test images using model.predict() method
practical_test_images =  test_images[:10]
prediction_probabilites = model.predict(practical_test_images)
prediction_probabilites

In [ ]:
# Altertive 1: Clean up model prediction using argmax to find the largest probablity
def derive_predicted_classes(prediction_probabilites):
    batch_prediction = []
    for vector in prediction_probabilites:
        batch_prediction.append(np.argmax(vector))
    return batch_prediction
    
model_prediction = derive_predicted_classes(prediction_probabilites)
model_prediction


In [ ]:
# Altertive 2: Get specific model preiction using the model.predict_classes() method
model_prediction = model.predict_classes(practical_test_images)
model_prediction

In [ ]:
np.array(fashion_mnist_class_names)[model_prediction]

In [ ]:
# Visualise the prediction result
plt.figure(figsize=(10,10))
for i in range(len(practical_test_images)):
    plt.subplot(5,5, i+1)
    plt.axis("off")
    plt.grid(False)
    plt.imshow(practical_test_images[i])
    plt.title(fashion_mnist_class_names[model_prediction[i]])
plt.show()

In [ ]:
model.save("image_classification_model.h5")

In [ ]:
# NOTE: If you encounter the error "AttributeError: 'str' object has no attribute 'decode'" when trying to load the model
# this is caused by using an higher version of h5py. Simply downgrade the package version of h5py.
# using pip "pip install 'h5py==2.10.0' --force-reinstall"
# or using conda "conda install -c anaconda h5py=2.10.0"
# reference: https://stackoverflow.com/questions/53740577/does-any-one-got-attributeerror-str-object-has-no-attribute-decode-whi

loaded_model = keras.models.load_model("image_classification_model.h5")
predictions = loaded_model.predict_classes(practical_test_images)
print(predictions)
print(np.array(fashion_mnist_class_names)[predictions])

In [ ]:
# Run Tensorboard using the command below from the location of the run folder
# tensorboard --logdir='runs'